# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f8054b58430>
├── 'a' --> tensor([[-0.1912,  0.8708,  0.5443],
│                   [-1.6953,  0.1030,  1.4793]])
└── 'x' --> <FastTreeValue 0x7f8054b58a60>
    └── 'c' --> tensor([[ 1.1593, -0.8663, -1.9790, -1.2510],
                        [-0.1363,  2.7561,  1.0063,  0.6106],
                        [ 0.3347,  0.9454,  0.0976, -0.2324]])

In [4]:
t.a

tensor([[-0.1912,  0.8708,  0.5443],
        [-1.6953,  0.1030,  1.4793]])

In [5]:
%timeit t.a

65.2 ns ± 0.112 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8054b58430>
├── 'a' --> tensor([[-0.2084,  0.6436,  0.6555],
│                   [-1.2319,  0.4082, -0.3411]])
└── 'x' --> <FastTreeValue 0x7f8054b58a60>
    └── 'c' --> tensor([[ 1.1593, -0.8663, -1.9790, -1.2510],
                        [-0.1363,  2.7561,  1.0063,  0.6106],
                        [ 0.3347,  0.9454,  0.0976, -0.2324]])

In [7]:
%timeit t.a = new_value

68.6 ns ± 0.0622 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1912,  0.8708,  0.5443],
               [-1.6953,  0.1030,  1.4793]]),
    x: Batch(
           c: tensor([[ 1.1593, -0.8663, -1.9790, -1.2510],
                      [-0.1363,  2.7561,  1.0063,  0.6106],
                      [ 0.3347,  0.9454,  0.0976, -0.2324]]),
       ),
)

In [10]:
b.a

tensor([[-0.1912,  0.8708,  0.5443],
        [-1.6953,  0.1030,  1.4793]])

In [11]:
%timeit b.a

58.8 ns ± 0.0358 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4150, -0.0538,  1.3016],
               [-0.6298, -0.7951,  1.1758]]),
    x: Batch(
           c: tensor([[ 1.1593, -0.8663, -1.9790, -1.2510],
                      [-0.1363,  2.7561,  1.0063,  0.6106],
                      [ 0.3347,  0.9454,  0.0976, -0.2324]]),
       ),
)

In [13]:
%timeit b.a = new_value

503 ns ± 0.199 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

814 ns ± 0.393 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.1 µs ± 15 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 154 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 340 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7fa4a52d00>
├── 'a' --> tensor([[[-0.1912,  0.8708,  0.5443],
│                    [-1.6953,  0.1030,  1.4793]],
│           
│                   [[-0.1912,  0.8708,  0.5443],
│                    [-1.6953,  0.1030,  1.4793]],
│           
│                   [[-0.1912,  0.8708,  0.5443],
│                    [-1.6953,  0.1030,  1.4793]],
│           
│                   [[-0.1912,  0.8708,  0.5443],
│                    [-1.6953,  0.1030,  1.4793]],
│           
│                   [[-0.1912,  0.8708,  0.5443],
│                    [-1.6953,  0.1030,  1.4793]],
│           
│                   [[-0.1912,  0.8708,  0.5443],
│                    [-1.6953,  0.1030,  1.4793]],
│           
│                   [[-0.1912,  0.8708,  0.5443],
│                    [-1.6953,  0.1030,  1.4793]],
│           
│                   [[-0.1912,  0.8708,  0.5443],
│                    [-1.6953,  0.1030,  1.4793]]])
└── 'x' --> <FastTreeValue 0x7f8054b54100>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 155 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7f9de85a90>
├── 'a' --> tensor([[-0.1912,  0.8708,  0.5443],
│                   [-1.6953,  0.1030,  1.4793],
│                   [-0.1912,  0.8708,  0.5443],
│                   [-1.6953,  0.1030,  1.4793],
│                   [-0.1912,  0.8708,  0.5443],
│                   [-1.6953,  0.1030,  1.4793],
│                   [-0.1912,  0.8708,  0.5443],
│                   [-1.6953,  0.1030,  1.4793],
│                   [-0.1912,  0.8708,  0.5443],
│                   [-1.6953,  0.1030,  1.4793],
│                   [-0.1912,  0.8708,  0.5443],
│                   [-1.6953,  0.1030,  1.4793],
│                   [-0.1912,  0.8708,  0.5443],
│                   [-1.6953,  0.1030,  1.4793],
│                   [-0.1912,  0.8708,  0.5443],
│                   [-1.6953,  0.1030,  1.4793]])
└── 'x' --> <FastTreeValue 0x7f7f9de85c40>
    └── 'c' --> tensor([[ 1.1593, -0.8663, -1.9790, -1.2510],
                        [-0.1363,  2.7561,  1.0063,  0.6106],
                 

In [23]:
%timeit t_cat(trees)

29.9 µs ± 77.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.8 µs ± 128 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.1593, -0.8663, -1.9790, -1.2510],
                       [-0.1363,  2.7561,  1.0063,  0.6106],
                       [ 0.3347,  0.9454,  0.0976, -0.2324]],
              
                      [[ 1.1593, -0.8663, -1.9790, -1.2510],
                       [-0.1363,  2.7561,  1.0063,  0.6106],
                       [ 0.3347,  0.9454,  0.0976, -0.2324]],
              
                      [[ 1.1593, -0.8663, -1.9790, -1.2510],
                       [-0.1363,  2.7561,  1.0063,  0.6106],
                       [ 0.3347,  0.9454,  0.0976, -0.2324]],
              
                      [[ 1.1593, -0.8663, -1.9790, -1.2510],
                       [-0.1363,  2.7561,  1.0063,  0.6106],
                       [ 0.3347,  0.9454,  0.0976, -0.2324]],
              
                      [[ 1.1593, -0.8663, -1.9790, -1.2510],
                       [-0.1363,  2.7561,  1.0063,  0.6106],
                       [ 0.3347,  0.9454,  0.0976, -0.2324]],

In [26]:
%timeit Batch.stack(batches)

77.8 µs ± 156 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.1593, -0.8663, -1.9790, -1.2510],
                      [-0.1363,  2.7561,  1.0063,  0.6106],
                      [ 0.3347,  0.9454,  0.0976, -0.2324],
                      [ 1.1593, -0.8663, -1.9790, -1.2510],
                      [-0.1363,  2.7561,  1.0063,  0.6106],
                      [ 0.3347,  0.9454,  0.0976, -0.2324],
                      [ 1.1593, -0.8663, -1.9790, -1.2510],
                      [-0.1363,  2.7561,  1.0063,  0.6106],
                      [ 0.3347,  0.9454,  0.0976, -0.2324],
                      [ 1.1593, -0.8663, -1.9790, -1.2510],
                      [-0.1363,  2.7561,  1.0063,  0.6106],
                      [ 0.3347,  0.9454,  0.0976, -0.2324],
                      [ 1.1593, -0.8663, -1.9790, -1.2510],
                      [-0.1363,  2.7561,  1.0063,  0.6106],
                      [ 0.3347,  0.9454,  0.0976, -0.2324],
                      [ 1.1593, -0.8663, -1.9790, -1.2510],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 319 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

337 µs ± 5.34 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
